# Alpaca-py stock trading basic

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alpacahq/alpaca-py/blob/master/examples/stocks/stocks-trading-basic.ipynb)

- This notebook shows how to use alpaca-py with stock trading API endpoints
- Please use ``paper account``. Please ``DO NOT`` use this notebook with live account. In this notebook, we place orders for stocks as an example.

In [1]:
# Please change the following to your own PAPER api key and secret
# or set them as environment variables (ALPACA_API_KEY, ALPACA_SECRET_KEY).
# You can get them from https://alpaca.markets/

api_key = None
secret_key = None

#### We use paper environment for this example ####
paper = True # Please do not modify this. This example is for paper trading only.
####

# Below are the variables for development this documents
# Please do not change these variables
trade_api_url = None
trade_api_wss = None
data_api_url = None
stream_data_wss = None

In [2]:
import os

for key, value in os.environ.items():
    print(f"{key}={value}")


ACSVCPORT=17532
ALLUSERSPROFILE=C:\ProgramData
APPDATA=C:\Users\garey\AppData\Roaming
CHOCOLATEYINSTALL=C:\ProgramData\chocolatey
CHOCOLATEYLASTPATHUPDATE=133770581846333310
COMMONPROGRAMFILES=C:\Program Files\Common Files
COMMONPROGRAMFILES(X86)=C:\Program Files (x86)\Common Files
COMMONPROGRAMW6432=C:\Program Files\Common Files
COMPUTERNAME=LAPTOP-6HKP9AF4
COMSPEC=C:\WINDOWS\system32\cmd.exe
CONDA_DEFAULT_ENV=alpaca
CONDA_EXE=C:\Users\garey\anaconda3\condabin\..\Scripts\conda.exe
CONDA_EXES="C:\Users\garey\anaconda3\condabin\..\Scripts\conda.exe"  
CONDA_PREFIX=C:\Users\garey\anaconda3\envs\alpaca
CONDA_PREFIX_1=C:\Users\garey\anaconda3
CONDA_PROMPT_MODIFIER=(alpaca) 
CONDA_PYTHON_EXE=C:\Users\garey\anaconda3\python.exe
CONDA_SHLVL=2
DRIVERDATA=C:\Windows\System32\Drivers\DriverData
EFC_12956_1592913036=1
GAMEFIRSTUSERPATH=C:\ProgramData\ASUS\GameFirst
GAMETURBO=C:\Program Files (x86)\ASUS\GameFirst\
HOMEDRIVE=C:
HOMEPATH=\Users\garey
IPY_INTERRUPT_EVENT=1956
JPY_INTERRUPT_EVENT=1956

In [14]:
import os

# Check if environment variables are set
api_key = os.getenv("ALPACA_API_KEY")
secret_key = os.getenv("ALPACA_SECRET_KEY")
trade_api_url = os.getenv("ALPACA_BASE_URL")
api_key



In [5]:
# install alpaca-py if it is not available
try:
    import alpaca
except ImportError:
    !python3 -m pip install alpaca-py
    import alpaca

In [6]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

from alpaca.trading.client import TradingClient
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.corporate_actions import CorporateActionsClient
from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.trading.stream import TradingStream
from alpaca.data.live.stock import StockDataStream

from alpaca.data.requests import (
    CorporateActionsRequest,
    StockBarsRequest,
    StockQuotesRequest,
    StockTradesRequest,
)
from alpaca.trading.requests import (
    ClosePositionRequest,
    GetAssetsRequest,
    GetOrdersRequest,
    LimitOrderRequest,
    MarketOrderRequest,
    StopLimitOrderRequest,
    StopLossRequest,
    StopOrderRequest,
    TakeProfitRequest,
    TrailingStopOrderRequest,
)
from alpaca.trading.enums import (
    AssetExchange,
    AssetStatus,
    OrderClass,
    OrderSide,
    OrderType,
    QueryOrderStatus,
    TimeInForce,
)

In [7]:
# to run async code in jupyter notebook
import nest_asyncio
nest_asyncio.apply()

In [8]:
# check version of alpaca-py
alpaca.__version__

'0.40.1'

# Trading Client

In [35]:
# setup clients
trade_client = TradingClient(api_key=api_key, secret_key=secret_key, paper=paper, url_override=trade_api_url)

In [36]:
# check trading account
# You can check definition of each field in the following documents
# ref. https://docs.alpaca.markets/docs/account-plans
# ref. https://docs.alpaca.markets/reference/getaccount-1
acct = trade_client.get_account()
acct

{   'account_blocked': False,
    'account_number': 'PA3GPJJBCV6D',
    'accrued_fees': '0',
    'buying_power': '173629.02',
    'cash': '73597.71',
    'created_at': datetime.datetime(2025, 6, 9, 15, 56, 50, 893330, tzinfo=TzInfo(UTC)),
    'crypto_status': <AccountStatus.ACTIVE: 'ACTIVE'>,
    'currency': 'USD',
    'daytrade_count': 4,
    'daytrading_buying_power': '0',
    'equity': '100031.31',
    'id': UUID('8273af54-8bd2-40e0-be96-857620779ffe'),
    'initial_margin': '13216.8',
    'last_equity': '99999.19622353472',
    'last_maintenance_margin': '3775.05',
    'long_market_value': '26433.6',
    'maintenance_margin': '7930.08',
    'multiplier': '2',
    'non_marginable_buying_power': '84814.5',
    'options_approved_level': 3,
    'options_buying_power': '84814.5',
    'options_trading_level': 3,
    'pattern_day_trader': False,
    'pending_transfer_in': None,
    'pending_transfer_out': None,
    'portfolio_value': '100031.31',
    'regt_buying_power': '173629.02',
    

In [37]:
# check account configuration
# ref. https://docs.alpaca.markets/reference/getaccountconfig-1
acct_config = trade_client.get_account_configurations()
acct_config

{   'dtbp_check': <DTBPCheck.ENTRY: 'entry'>,
    'fractional_trading': True,
    'max_margin_multiplier': '4',
    'max_options_trading_level': None,
    'no_shorting': False,
    'pdt_check': <PDTCheck.ENTRY: 'entry'>,
    'ptp_no_exception_entry': False,
    'suspend_trade': False,
    'trade_confirm_email': <TradeConfirmationEmail.ALL: 'all'>}

In [38]:
# set account configuration
# ref. https://docs.alpaca.markets/reference/patchaccountconfig-1
req = acct_config
req.fractional_trading = not req.fractional_trading # toggle fractional trading
acct_config_new = trade_client.set_account_configurations(req)
display(acct_config_new)

# revert changes
req = acct_config_new
req.fractional_trading = not req.fractional_trading # toggle fractional trading
acct_config_reverted = trade_client.set_account_configurations(req)
display(acct_config_reverted)

{   'dtbp_check': <DTBPCheck.ENTRY: 'entry'>,
    'fractional_trading': False,
    'max_margin_multiplier': '4',
    'max_options_trading_level': None,
    'no_shorting': False,
    'pdt_check': <PDTCheck.ENTRY: 'entry'>,
    'ptp_no_exception_entry': False,
    'suspend_trade': False,
    'trade_confirm_email': <TradeConfirmationEmail.ALL: 'all'>}

{   'dtbp_check': <DTBPCheck.ENTRY: 'entry'>,
    'fractional_trading': True,
    'max_margin_multiplier': '4',
    'max_options_trading_level': None,
    'no_shorting': False,
    'pdt_check': <PDTCheck.ENTRY: 'entry'>,
    'ptp_no_exception_entry': False,
    'suspend_trade': False,
    'trade_confirm_email': <TradeConfirmationEmail.ALL: 'all'>}

In [39]:
# get list of assets which are us_equity (default), active, and in NASDAQ
# ref. https://docs.alpaca.markets/reference/get-v2-assets-1
req = GetAssetsRequest(
  # asset_class=AssetClass.US_EQUITY,  # default asset_class is us_equity
  status=AssetStatus.ACTIVE,
  exchange=AssetExchange.NASDAQ,
)
assets = trade_client.get_all_assets(req)
assets[:2]

[{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'attributes': [],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.NASDAQ: 'NASDAQ'>,
     'fractionable': False,
     'id': UUID('7af0066a-e334-4de7-8222-9a75376a5766'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': None,
     'min_trade_increment': None,
     'name': 'E-Home Household Service Holdings Limited Ordinary Shares',
     'price_increment': None,
     'shortable': False,
     'status': <AssetStatus.ACTIVE: 'active'>,
     'symbol': 'EJH',
     'tradable': False},
 {   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'attributes': ['has_options'],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.NASDAQ: 'NASDAQ'>,
     'fractionable': False,
     'id': UUID('cc492b87-aff1-442a-99bb-e4e638dd6e98'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': None,
     'min_trade_increment': None,
  

### Orders

In [40]:
# we will place orders which Alapca trading platform supports
# - order classes: simple, bracket, oco, oto
# - order types: market, limit, stop, stop_limit, trailing_stop
#
# please refer to the following documents for more details
# ref. https://docs.alpaca.markets/docs/orders-at-alpaca
# ref. https://docs.alpaca.markets/reference/postorder
#
# we will also use fractional trading capability of Alpaca trading platform in this example
# ref. https://docs.alpaca.markets/docs/fractional-trading

# we will place orders for symbol: SPY in this example
symbol = "SPY"

In [41]:
# simple, market order, fractional qty
# Alpaca trading platform support fractional trading by default
# you can specify:
# fractional qty (e.g. 0.01 qty) in the order request (which is shown in this example)
# or notional value (e.g. 100 USD) (which is in the next example)
#
# If you have an error of `qty must be integer`,
# please try to `Reset Account` of your paper account via the Alpaca Trading API dashboard
req = MarketOrderRequest(
    symbol = symbol,
    qty = 5.5, # Example: place an order for $100 dollars worth of SPY
    #notional = 100, # Example: place an order for 5.5 shares of SPY
    side = OrderSide.BUY,
    type = OrderType.MARKET,
    time_in_force = TimeInForce.DAY,
)
res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': '8249112b-e2c4-4529-b67d-31ac58d507b9',
    'created_at': datetime.datetime(2025, 6, 10, 15, 9, 10, 468752, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': datetime.datetime(2025, 6, 10, 20, 0, tzinfo=TzInfo(UTC)),
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('51bfa611-3790-4f7b-b4b1-06421650b9fb'),
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.MARKET: 'market'>,
    'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
    'qty': '5.5',
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderSta

In [42]:
# simple, market order, notional
# Alpaca trading platform support fractional trading by default
# you can specify:
# fractional qty (e.g. 0.01 qty) in the order request (which is in the example above)
# or notional value (e.g. 100 USD) (which is in this example)
req = MarketOrderRequest(
    symbol = symbol,
    notional = 1.11,  # notional is specified in USD, here we specify $1.11
    side = OrderSide.BUY,
    type = OrderType.MARKET,
    time_in_force = TimeInForce.DAY,
)
res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': 'f7a3da8f-cebf-4eb2-82af-03bd0bd08c96',
    'created_at': datetime.datetime(2025, 6, 10, 15, 9, 13, 588835, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': datetime.datetime(2025, 6, 10, 20, 0, tzinfo=TzInfo(UTC)),
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('bf233c4a-6d55-4661-adcf-80cb4712b6c1'),
    'legs': None,
    'limit_price': None,
    'notional': '1.11',
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.MARKET: 'market'>,
    'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
    'qty': None,
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderSt

In [43]:
# simple, limit order, fractional qty
req = LimitOrderRequest(
    symbol = symbol,
    qty = 0.01,
    limit_price = 550.25,
    side = OrderSide.BUY,
    type = OrderType.LIMIT,
    time_in_force = TimeInForce.DAY,
)
res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': '9a08584d-8431-4a86-a405-178f88ce6543',
    'created_at': datetime.datetime(2025, 6, 10, 15, 9, 15, 770685, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': datetime.datetime(2025, 6, 10, 20, 0, tzinfo=TzInfo(UTC)),
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('2eb64d9e-bcad-49fa-a8d8-5484d9a86de5'),
    'legs': None,
    'limit_price': '550.25',
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.LIMIT: 'limit'>,
    'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
    'qty': '0.01',
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <Order

In [44]:
# stop order
req = StopOrderRequest(
                    symbol = symbol,
                    qty = 1,
                    side = OrderSide.BUY,
                    time_in_force = TimeInForce.GTC,
                    stop_price = 600
                    )

res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': '1f27e2c9-5401-49f8-9eca-ca2a47c9de0d',
    'created_at': datetime.datetime(2025, 6, 10, 15, 9, 18, 87579, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': datetime.datetime(2025, 9, 8, 20, 0, tzinfo=TzInfo(UTC)),
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('8626df7b-d4b0-40f1-a569-69c085ff3661'),
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.STOP: 'stop'>,
    'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
    'qty': '1',
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.PEND

In [45]:
# stop limit order
req = StopLimitOrderRequest(
                    symbol = symbol,
                    qty = 1,
                    side = OrderSide.BUY,
                    time_in_force = TimeInForce.GTC,
                    limit_price = 550,
                    stop_price = 600
                    )

res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': '28285c95-f015-4380-8478-f9129cdf689b',
    'created_at': datetime.datetime(2025, 6, 10, 15, 9, 20, 340225, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': datetime.datetime(2025, 9, 8, 20, 0, tzinfo=TzInfo(UTC)),
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('8e024c3a-754e-44ea-870d-2b1dd983161c'),
    'legs': None,
    'limit_price': '550',
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.STOP_LIMIT: 'stop_limit'>,
    'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
    'qty': '1',
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <Or

In [47]:
# bracket order with both stop loss and take profit
req = MarketOrderRequest(
                    symbol = symbol,
                    qty = 5,
                    side = OrderSide.BUY,
                    time_in_force = TimeInForce.DAY,
                    order_class = OrderClass.BRACKET,
                    take_profit = TakeProfitRequest(limit_price=602),
                    stop_loss = StopLossRequest(stop_price=300)
)
res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': 'ff92faa7-3daa-4e01-96ab-5c5de13c3eb5',
    'created_at': datetime.datetime(2025, 6, 10, 15, 9, 41, 422920, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': datetime.datetime(2025, 6, 10, 20, 0, tzinfo=TzInfo(UTC)),
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('a2c304a2-dd98-40b9-8b04-5afc88465473'),
    'legs': [   {   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
                    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
                    'canceled_at': None,
                    'client_order_id': 'cb31ed08-9eac-45e6-b9bf-0a55c442440a',
                    'created_at': datetime.datetime(2025, 6, 10, 15, 9, 41, 422920, tzinfo=TzInfo(UTC)),
                    'expired

In [48]:
# oto order with stop loss
req = LimitOrderRequest(
                    symbol = symbol,
                    qty = 1,
                    limit_price = 500,
                    side = OrderSide.BUY,
                    time_in_force = TimeInForce.DAY,
                    Class = OrderClass.OTO,
                    stop_loss = StopLossRequest(stop_price = 300)
                    )

res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': '3187cc7b-60db-4a46-8f2a-5232ce2348ae',
    'created_at': datetime.datetime(2025, 6, 10, 15, 9, 43, 728153, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': datetime.datetime(2025, 6, 10, 20, 0, tzinfo=TzInfo(UTC)),
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('457d33cb-5a2b-43ec-9b3c-08d4c9473038'),
    'legs': None,
    'limit_price': '500',
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.LIMIT: 'limit'>,
    'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
    'qty': '1',
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus

In [49]:
# oco limit order
req = LimitOrderRequest(
                    symbol = symbol,
                    qty = 1,
                    limit_price = 500,
                    side = OrderSide.BUY,
                    time_in_force = TimeInForce.DAY,
                    Class = OrderClass.OCO
                    )

res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': '2db84833-b99c-4c3d-8897-4c2d11f894e0',
    'created_at': datetime.datetime(2025, 6, 10, 15, 9, 45, 983058, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': datetime.datetime(2025, 6, 10, 20, 0, tzinfo=TzInfo(UTC)),
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('6a1cef49-f444-4bfe-8044-009aed269586'),
    'legs': None,
    'limit_price': '500',
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.LIMIT: 'limit'>,
    'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
    'qty': '1',
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus

In [50]:
# trailing stop order
req = TrailingStopOrderRequest(
                    symbol = symbol,
                    qty = 1,
                    side = OrderSide.SELL,
                    time_in_force = TimeInForce.GTC,
                    trail_percent = 0.20 # you can also use trail_price instead of trail_percent
                    )

res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': '3b4d2258-18f0-4132-b174-dd6edf134892',
    'created_at': datetime.datetime(2025, 6, 10, 15, 9, 48, 80094, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': datetime.datetime(2025, 9, 8, 20, 0, tzinfo=TzInfo(UTC)),
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': '601.03',
    'id': UUID('b0d6e3cb-0b35-439f-a80e-4782dfd995ff'),
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.TRAILING_STOP: 'trailing_stop'>,
    'position_intent': <PositionIntent.SELL_TO_CLOSE: 'sell_to_close'>,
    'qty': '1',
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.SELL: 'sell'>,
   

In [51]:
# get a list of orders including closed (e.g. filled) orders by specifying symbol
req = GetOrdersRequest(
    status = QueryOrderStatus.ALL,
    symbols = [symbol]
)
orders = trade_client.get_orders(req)
orders

[{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
     'canceled_at': None,
     'client_order_id': '3b4d2258-18f0-4132-b174-dd6edf134892',
     'created_at': datetime.datetime(2025, 6, 10, 15, 9, 48, 80094, tzinfo=TzInfo(UTC)),
     'expired_at': None,
     'expires_at': datetime.datetime(2025, 9, 8, 20, 0, tzinfo=TzInfo(UTC)),
     'extended_hours': False,
     'failed_at': None,
     'filled_at': None,
     'filled_avg_price': None,
     'filled_qty': '0',
     'hwm': '601.03',
     'id': UUID('b0d6e3cb-0b35-439f-a80e-4782dfd995ff'),
     'legs': None,
     'limit_price': None,
     'notional': None,
     'order_class': <OrderClass.SIMPLE: 'simple'>,
     'order_type': <OrderType.TRAILING_STOP: 'trailing_stop'>,
     'position_intent': <PositionIntent.SELL_TO_CLOSE: 'sell_to_close'>,
     'qty': '1',
     'ratio_qty': None,
     'replaced_at': None,
     'replaced_by': None,
     'replaces': None,
     'side': <Or

In [52]:
# see all open orders
req = GetOrdersRequest(
    status = QueryOrderStatus.OPEN,
    symbols = [symbol]
)
open_orders = trade_client.get_orders(req)
open_orders

[{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
     'canceled_at': None,
     'client_order_id': '3b4d2258-18f0-4132-b174-dd6edf134892',
     'created_at': datetime.datetime(2025, 6, 10, 15, 9, 48, 80094, tzinfo=TzInfo(UTC)),
     'expired_at': None,
     'expires_at': datetime.datetime(2025, 9, 8, 20, 0, tzinfo=TzInfo(UTC)),
     'extended_hours': False,
     'failed_at': None,
     'filled_at': None,
     'filled_avg_price': None,
     'filled_qty': '0',
     'hwm': '601.03',
     'id': UUID('b0d6e3cb-0b35-439f-a80e-4782dfd995ff'),
     'legs': None,
     'limit_price': None,
     'notional': None,
     'order_class': <OrderClass.SIMPLE: 'simple'>,
     'order_type': <OrderType.TRAILING_STOP: 'trailing_stop'>,
     'position_intent': <PositionIntent.SELL_TO_CLOSE: 'sell_to_close'>,
     'qty': '1',
     'ratio_qty': None,
     'replaced_at': None,
     'replaced_by': None,
     'replaces': None,
     'side': <Or

In [53]:
# cancel all open orders
trade_client.cancel_orders()

[{   'body': None,
     'id': UUID('2eb64d9e-bcad-49fa-a8d8-5484d9a86de5'),
     'status': 200},
 {   'body': None,
     'id': UUID('457d33cb-5a2b-43ec-9b3c-08d4c9473038'),
     'status': 200},
 {   'body': None,
     'id': UUID('6a1cef49-f444-4bfe-8044-009aed269586'),
     'status': 200},
 {   'body': None,
     'id': UUID('7160d799-e994-4591-8570-76a7774af31c'),
     'status': 200},
 {   'body': None,
     'id': UUID('8e024c3a-754e-44ea-870d-2b1dd983161c'),
     'status': 200},
 {   'body': None,
     'id': UUID('b0d6e3cb-0b35-439f-a80e-4782dfd995ff'),
     'status': 200}]

### Positions

In [54]:
# get all open positions
# ref. https://docs.alpaca.markets/reference/getallopenpositions-1
positions = trade_client.get_all_positions()
positions

[{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
     'asset_marginable': True,
     'avg_entry_price': '600.63194',
     'avg_entry_swap_rate': None,
     'change_today': '0.0022658751334045',
     'cost_basis': '33316.547527',
     'current_price': '601.0388',
     'exchange': <AssetExchange.ARCA: 'ARCA'>,
     'lastday_price': '599.68',
     'market_value': '33339.115709',
     'qty': '55.469157247',
     'qty_available': '55.469157247',
     'side': <PositionSide.LONG: 'long'>,
     'swap_rate': None,
     'symbol': 'SPY',
     'unrealized_intraday_pl': '75.371491',
     'unrealized_intraday_plpc': '0.002265875137388',
     'unrealized_pl': '22.568182',
     'unrealized_plpc': '0.0006773865743955',
     'usd': None}]

In [55]:
# get positions by symbol
# ref. https://docs.alpaca.markets/reference/getopenposition-1
position = trade_client.get_open_position(symbol_or_asset_id=symbol)
position


{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'asset_marginable': True,
    'avg_entry_price': '600.63194',
    'avg_entry_swap_rate': None,
    'change_today': '0.0022678762006403',
    'cost_basis': '33316.547527',
    'current_price': '601.04',
    'exchange': <AssetExchange.ARCA: 'ARCA'>,
    'lastday_price': '599.68',
    'market_value': '33339.182272',
    'qty': '55.469157247',
    'qty_available': '55.469157247',
    'side': <PositionSide.LONG: 'long'>,
    'swap_rate': None,
    'symbol': 'SPY',
    'unrealized_intraday_pl': '75.438054',
    'unrealized_intraday_plpc': '0.0022678762049637',
    'unrealized_pl': '22.634745',
    'unrealized_plpc': '0.0006793844704844',
    'usd': None}

In [56]:
# get positions by asset_id
trade_client.get_open_position(symbol_or_asset_id=position.asset_id)

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'asset_marginable': True,
    'avg_entry_price': '600.63194',
    'avg_entry_swap_rate': None,
    'change_today': '0.0022678762006403',
    'cost_basis': '33316.547527',
    'current_price': '601.04',
    'exchange': <AssetExchange.ARCA: 'ARCA'>,
    'lastday_price': '599.68',
    'market_value': '33339.182272',
    'qty': '55.469157247',
    'qty_available': '55.469157247',
    'side': <PositionSide.LONG: 'long'>,
    'swap_rate': None,
    'symbol': 'SPY',
    'unrealized_intraday_pl': '75.438054',
    'unrealized_intraday_plpc': '0.0022678762049637',
    'unrealized_pl': '22.634745',
    'unrealized_plpc': '0.0006793844704844',
    'usd': None}

In [57]:
# close the position with specifying qty
# ref. https://docs.alpaca.markets/reference/deleteopenposition-1
trade_client.close_position(
    symbol_or_asset_id = symbol,
    close_options = ClosePositionRequest(
        qty = "0.01",
    )
)

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'canceled_at': None,
    'client_order_id': 'c3164267-d415-4e97-bd25-d8015706b84c',
    'created_at': datetime.datetime(2025, 6, 10, 15, 10, 1, 56654, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': datetime.datetime(2025, 6, 10, 20, 0, tzinfo=TzInfo(UTC)),
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('d5558eb4-159b-4622-b557-a2cc99ab3476'),
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.MARKET: 'market'>,
    'position_intent': <PositionIntent.SELL_TO_CLOSE: 'sell_to_close'>,
    'qty': '0.01',
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.SELL: 'sell'>,
    'status': <Or

# Trade Update (Stream)

With TradingStream client, you can get updates about trades

You can open this notebook in another window and run below cell to check trade updates.

In [ ]:
# subscribe trade updates
trade_stream_client = TradingStream(api_key, secret_key, paper=paper, url_override = trade_api_wss)

async def trade_updates_handler(data):
    print(data)

trade_stream_client.subscribe_trade_updates(trade_updates_handler)
trade_stream_client.run()

# Market Data (Historical)

In [59]:
# setup stock historical data client
stock_historical_data_client = StockHistoricalDataClient(api_key, secret_key, url_override = data_api_url)

In [60]:
# get historical bars by symbol
# ref. https://docs.alpaca.markets/reference/stockbars-1
now = datetime.now(ZoneInfo("America/New_York"))
req = StockBarsRequest(
    symbol_or_symbols = [symbol],
    timeframe=TimeFrame(amount = 1, unit = TimeFrameUnit.Hour), # specify timeframe
    start = now - timedelta(days = 5),                          # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                        # specify end datetime, default=now
    limit = 2,                                               # specify limit
)
stock_historical_data_client.get_stock_bars(req).df

open      high      low   close  \
symbol timestamp                                                      
SPY    2025-06-05 16:00:00+00:00  598.76  598.9500  594.260  594.76   
       2025-06-05 17:00:00+00:00  594.75  596.9797  594.305  595.82   

                                     volume  trade_count        vwap  
symbol timestamp                                                      
SPY    2025-06-05 16:00:00+00:00  9659760.0     113349.0  596.676118  
       2025-06-05 17:00:00+00:00  6908398.0      82166.0  595.978784

In [61]:
# get historical trades by symbol
req = StockTradesRequest(
    symbol_or_symbols = [symbol],
    start = now - timedelta(days = 5),                          # specify start datetime, default=the beginning of the current day.
    # end=None,                                             # specify end datetime, default=now
    limit = 2,                                                # specify limit
)
stock_historical_data_client.get_stock_trades(req).df

exchange     price  size  \
symbol timestamp                                                   
SPY    2025-06-05 15:10:11.842778+00:00        D  597.4801   4.0   
       2025-06-05 15:10:12.000359+00:00        Z  597.4900  10.0   

                                                     id conditions tape  
symbol timestamp                                                         
SPY    2025-06-05 15:10:11.842778+00:00  71688459998788     [ , I]    B  
       2025-06-05 15:10:12.000359+00:00  52983525886834     [ , I]    B

In [62]:
# get historical quotes by symbol
req = StockQuotesRequest(
    symbol_or_symbols = [symbol],
    start = now - timedelta(days = 5),                      # specify start datetime, default=the beginning of the current day.
    # end=None,                                             # specify end datetime, default=now
    limit = 2,                                              # specify limit
)
stock_historical_data_client.get_stock_quotes(req).df

bid_price  bid_size bid_exchange  \
symbol timestamp                                                            
SPY    2025-06-05 15:10:11.750915+00:00     597.48       3.0            P   
       2025-06-05 15:10:11.848998+00:00     597.48       2.0            P   

                                         ask_price  ask_size ask_exchange  \
symbol timestamp                                                            
SPY    2025-06-05 15:10:11.750915+00:00     597.49       2.0            T   
       2025-06-05 15:10:11.848998+00:00     597.49       2.0            T   

                                        conditions tape  
symbol timestamp                                         
SPY    2025-06-05 15:10:11.750915+00:00        [R]    B  
       2025-06-05 15:10:11.848998+00:00        [R]    B

In [63]:
# get latest quotes by symbol
req = StockQuotesRequest(
    symbol_or_symbols = [symbol],
)
res = stock_historical_data_client.get_stock_latest_quote(req)
res

{'SPY': {   'ask_exchange': 'V',
     'ask_price': 600.91,
     'ask_size': 1.0,
     'bid_exchange': 'V',
     'bid_price': 600.88,
     'bid_size': 1.0,
     'conditions': ['R'],
     'symbol': 'SPY',
     'tape': 'B',
     'timestamp': datetime.datetime(2025, 6, 10, 15, 10, 16, 903592, tzinfo=TzInfo(UTC))}}

# Market Data (Stream)

In [64]:
stock_data_stream_client = StockDataStream(api_key, secret_key, url_override = stream_data_wss)

async def stock_data_stream_handler(data):
    print(data)

symbols = [symbol]

stock_data_stream_client.subscribe_quotes(stock_data_stream_handler, *symbols)
stock_data_stream_client.subscribe_trades(stock_data_stream_handler, *symbols)

stock_data_stream_client.run()

symbol='SPY' timestamp=datetime.datetime(2025, 6, 10, 15, 10, 19, 229677, tzinfo=datetime.timezone.utc) bid_price=600.92 bid_size=1.0 bid_exchange='V' ask_price=600.94 ask_size=1.0 ask_exchange='V' conditions=['R'] tape='B'
symbol='SPY' timestamp=datetime.datetime(2025, 6, 10, 15, 10, 19, 319646, tzinfo=datetime.timezone.utc) bid_price=600.92 bid_size=1.0 bid_exchange='V' ask_price=600.94 ask_size=2.0 ask_exchange='V' conditions=['R'] tape='B'
symbol='SPY' timestamp=datetime.datetime(2025, 6, 10, 15, 10, 19, 365992, tzinfo=datetime.timezone.utc) bid_price=600.92 bid_size=1.0 bid_exchange='V' ask_price=600.94 ask_size=1.0 ask_exchange='V' conditions=['R'] tape='B'
symbol='SPY' timestamp=datetime.datetime(2025, 6, 10, 15, 10, 19, 372207, tzinfo=datetime.timezone.utc) exchange='V' price=600.94 size=100.0 id=52983594698351 conditions=[' '] tape='B'
symbol='SPY' timestamp=datetime.datetime(2025, 6, 10, 15, 10, 19, 372226, tzinfo=datetime.timezone.utc) bid_price=600.92 bid_size=1.0 bid_excha

TimeoutError: 

# Corporate actions

In [65]:
corporate_actions_client = CorporateActionsClient(api_key, secret_key)
corporate_actions_client.get_corporate_actions(CorporateActionsRequest(
    start=datetime(2020, 1, 1),
    symbols=[symbol]
)).df

,symbol,rate,special,foreign,process_date,ex_date,record_date,payable_date
corporate_action_type,,,,,,,,
cash_dividends,SPY,1.405559,False,False,2020-04-30,2020-03-20,2020-03-23,2020-04-30
cash_dividends,SPY,1.366242,False,False,2020-07-31,2020-06-19,2020-06-22,2020-07-31
cash_dividends,SPY,1.339224,False,False,2020-10-30,2020-09-18,2020-09-21,2020-10-30
cash_dividends,SPY,1.580000,False,False,2021-01-29,2020-12-18,2020-12-21,2021-01-29
cash_dividends,SPY,1.277788,False,False,2021-04-30,2021-03-19,2021-03-22,2021-04-30
cash_dividends,SPY,1.375875,False,False,2021-07-30,2021-06-18,2021-06-21,2021-07-30
cash_dividends,SPY,1.428117,False,False,2021-10-29,2021-09-17,2021-09-20,2021-10-29
cash_dividends,SPY,1.636431,False,False,2022-01-31,2021-12-17,2021-12-20,2022-01-31
cash_dividends,SPY,1.366009,False,False,2022-04-29,2022-03-18,2022-03-21,2022-04-29
